Use Optuna to optimize the hyperparameters of the projection head for HAR.

- architecture: N layers (1 to 3), N neurons (4 to 256), dropout (0 to 0.5)
- optimizer: (AdamW, Adam, RMSprop, SGD), lr
- others: batch size

look at the difference wrt AdamW

In [1]:
import torch as t
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader#, #Dataset, 
import torch.optim as optim

#from torch.nn import Module, Sequential, Conv2d, BatchNorm2d, ReLU, MaxPool2d, AvgPool2d, Linear, Dropout, ZeroPad2d, CrossEntropyLoss
#from torch.optim import SGD, Adam, AdamW, lr_scheduler
#from tqdm import tqdm

import optuna
from optuna.trial import TrialState
from optuna.visualization import plot_param_importances
import plotly
from plotly.io import show

from utils_preprocess import *
from utils_traintest import *
from models.ResNet18 import ResNet18

seed = 203345656
torch.manual_seed(seed)

In [3]:
# define the model to optimize
# a FFNN, optimize the number of layers, hidden units, dropout 
def define_model(trial, num_classes):
    n_layers = trial.suggest_int("n_layers", 1, 3)
    layers = []

    in_features = 2560
    for i in range(n_layers):
        out_features = trial.suggest_int("n_units_l{}".format(i), 4, 256)
        layers.append(nn.Linear(in_features, out_features))
        layers.append(nn.ReLU(inplace=True))
        p = trial.suggest_float("dropout_l{}".format(i), 0, 0.5)
        layers.append(nn.Dropout(p))

        in_features = out_features
    layers.append(nn.Linear(in_features, num_classes))

    return nn.Sequential(*layers)

# define the objective function to maximize (or minimize)
# in our case is the accuracy on the validation set
def objective(trial, **kwargs):
    
    num_classes = kwargs['num_classes']
    train_feats_data = kwargs['train_feats']
    val_feats_data = kwargs['val_feats']
    batch_size = kwargs['batch_size']
    device = kwargs['device']
    epochs = kwargs['epochs']
    
    # Generate the model
    model = define_model(trial, num_classes).to(device)

    # generate the optimizers
    optimizer_name = trial.suggest_categorical("optimizer", ["Adam", "AdamW"])    
    lr = trial.suggest_float("lr", 1e-5, 1e-1, log=True)
    optimizer = getattr(optim, optimizer_name)(model.parameters(), lr=lr)

    train_loader = DataLoader(train_feats_data, batch_size=batch_size, shuffle=True, drop_last=False)
    val_loader = DataLoader(val_feats_data, batch_size=batch_size, shuffle=False, drop_last=False)

    for epoch in range(epochs):
        model.train()
        for batch in train_loader:
            feats, labels = batch
            feats = feats.to(device)
            labels = labels.to(device)
            labels = labels.squeeze(dim=1)
            preds = model(feats)
            loss = F.cross_entropy(preds, labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        model.eval()
        with torch.no_grad():
            vaccs = []
            model.eval()
            for batch in val_loader:
                feats, labels = batch
                feats = feats.to(device)
                labels = labels.to(device)
                labels = labels.squeeze(dim=1)
                preds = model(feats)
                acc = (preds.argmax(dim=-1) == labels).float().mean()
                vaccs.append(acc.item())

            val_acc = np.mean(vaccs)
        
        trial.report(val_acc, epoch)

        # Handle pruning based on the intermediate value.
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()

    return val_acc

In [ ]:
# call the trained encoder
simclr_model = ResNet18(hidden_dimension=16)
path_model = 'models/ResNet18/My_ResNet18_hd16_bs128.ckpt'
device = 'cuda' if torch.cuda.is_available() else 'cpu'
simclr_model.load_state_dict(torch.load(path_model, map_location=torch.device(device), weights_only=True))

#### HAR projection head optimization

In [ ]:
# prepare the training and validation data
num_classes = 5
dir_init = './doppler_traces/' #Directory of data
subdirs_init = 'S1a,S1b,S1c' #Subdirs for training

dataset_csi_train_class = create_training_set(dir_init=dir_init, subdirs_init=subdirs_init)
dataset_csi_val_class = create_validation_set(dir_init=dir_init, subdirs_init=subdirs_init)

# prepare the features
train_feats = prepare_data_features(simclr_model, dataset_csi_train_class, device=device)
val_feats = prepare_data_features(simclr_model, dataset_csi_val_class, device=device)

In [ ]:
batch_size = 64 # may optimize that as well...
epochs = 10

kwargs = {'num_classes': num_classes,
          'train_feats':train_feats,
          'val_feats': val_feats,
          'device': device,
          'batch_size': batch_size,
          'epochs': epochs}

# run the optimizer
study = optuna.create_study(direction="maximize")
study.optimize(lambda trial: objective(trial, **kwargs), n_trials=10, timeout=600)

pruned_trials = study.get_trials(deepcopy=False, states=[TrialState.PRUNED])
complete_trials = study.get_trials(deepcopy=False, states=[TrialState.COMPLETE])

print("Study statistics: ")
print("  Number of finished trials: ", len(study.trials))
print("  Number of pruned trials: ", len(pruned_trials))
print("  Number of complete trials: ", len(complete_trials))

print("Best trial:")
trial = study.best_trial

print("  Value: ", trial.value)

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

fig = plot_param_importances(study)
show(fig)

[I 2025-01-12 16:03:13,698] A new study created in memory with name: no-name-15f42a35-d8e9-4b34-acc9-df4faddb0570
[I 2025-01-12 16:03:21,265] Trial 0 finished with value: 0.6495535714285714 and parameters: {'n_layers': 2, 'n_units_l0': 138, 'dropout_l0': 0.46657928738397914, 'n_units_l1': 213, 'dropout_l1': 0.19782568174112125, 'optimizer': 'Adam', 'lr': 0.016066052663014866}. Best is trial 0 with value: 0.6495535714285714.
[I 2025-01-12 16:03:25,750] Trial 1 finished with value: 0.8221130950110299 and parameters: {'n_layers': 1, 'n_units_l0': 118, 'dropout_l0': 0.2995020868593735, 'optimizer': 'AdamW', 'lr': 0.004391559942566321}. Best is trial 1 with value: 0.8221130950110299.
[I 2025-01-12 16:03:31,950] Trial 2 finished with value: 0.20357142857142857 and parameters: {'n_layers': 3, 'n_units_l0': 126, 'dropout_l0': 0.1841574669097248, 'n_units_l1': 30, 'dropout_l1': 0.31881937161926244, 'n_units_l2': 128, 'dropout_l2': 0.12068684556807063, 'optimizer': 'Adam', 'lr': 0.04571496822308

Study statistics: 
  Number of finished trials:  10
  Number of pruned trials:  3
  Number of complete trials:  7
Best trial:
  Value:  0.8719345246042524
  Params: 
    n_layers: 1
    n_units_l0: 120
    dropout_l0: 0.2630538939516411
    optimizer: Adam
    lr: 0.0012429680599979171


#### PI projection head optimization

In [ ]:
# prepare the training and validation data
num_classes = 3
dir_init = './doppler_traces/' #Directory of data
subdirs_init = 'S1a,S3a,S7a' #Subdirs for training

dataset_PI_train_class = create_training_set(dir_init=dir_init, subdirs_init=subdirs_init, PI=num_classes)
dataset_PI_val_class = create_validation_set(dir_init=dir_init, subdirs_init=subdirs_init, PI=num_classes)

# prepare the features
train_feats_PI = prepare_data_features(simclr_model, dataset_csi_train_class, device=device)
val_feats_PI = prepare_data_features(simclr_model, dataset_csi_val_class, device=device)

In [ ]:
batch_size = 64 # may optimize that as well...
epochs = 10

kwargs = {'num_classes': num_classes,
          'train_feats':train_feats,
          'val_feats': val_feats,
          'device': device,
          'batch_size': batch_size,
          'epochs': epochs}

# run the optimizer
study = optuna.create_study(direction="maximize")
study.optimize(lambda trial: objective(trial, **kwargs), n_trials=10, timeout=600)

pruned_trials = study.get_trials(deepcopy=False, states=[TrialState.PRUNED])
complete_trials = study.get_trials(deepcopy=False, states=[TrialState.COMPLETE])

print("Study statistics: ")
print("  Number of finished trials: ", len(study.trials))
print("  Number of pruned trials: ", len(pruned_trials))
print("  Number of complete trials: ", len(complete_trials))

print("Best trial:")
trial = study.best_trial

print("  Value: ", trial.value)

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

fig = plot_param_importances(study)
show(fig)